<a href="https://colab.research.google.com/github/tam1444AH/Finetuning-Qwen3/blob/main/notebooks/codeLLMFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["WANDB_DISABLED"] = "false"
import wandb
import shutil
from datetime import datetime
%pip install nbstripout

REPO_URL="https://github.com/tam1444AH/Finetuning-Qwen3.git"
REPO="Finetuning-Qwen3"

os.chdir("/content")

# If repo exists, update it; otherwise, clone fresh
if os.path.exists(REPO):
    print(f"Repo '{REPO}' exists, pulling latest changes...")
    os.chdir(REPO)
    !git reset --hard HEAD   # optional: discard local changes
    !git pull
else:
    print(f"Cloning repo '{REPO}'...")
    !git clone "$REPO_URL" "$REPO"
    os.chdir(REPO)

!nbstripout --install
!git branch -a


# Install dependencies if present
if os.path.exists("requirements.txt"):
    !pip install -r requirements.txt
if os.path.exists("pyproject.toml"):
    !pip install -e .

Cloning repo 'Finetuning-Qwen3'...
Cloning into 'Finetuning-Qwen3'...
remote: Enumerating objects: 521, done.
remote: Counting objects: 100% (247/247), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 521 (delta 192), reused 155 (delta 149), pack-reused 274 (from 1)
Receiving objects: 100% (521/521), 1.62 MiB | 2.96 MiB/s, done.
Resolving deltas: 100% (297/297), done.
* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/main
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 79.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 20.6 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [2]:
!pip -q install -U huggingface_hub hf_transfer
!export HF_HUB_ENABLE_HF_TRANSFER=1

import os
from google.colab import userdata
from huggingface_hub import login, whoami

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # mitigate fragmentation
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
HF_TOKEN = userdata.get('HF_TOKEN')
WANDB_TOKEN = userdata.get('WANDB_KEY')
os.environ["WANDB_API_KEY"] = WANDB_TOKEN
os.environ["HF_TOKEN"] = HF_TOKEN
wandb.login(key=WANDB_TOKEN, relogin=True)
login(token=HF_TOKEN, add_to_git_credential=True)  # also sets Git creds for LFS

print("Logged in as:", whoami(token=HF_TOKEN)["name"])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 23.5 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammedtalha290 (mohammedtalha290-university-of-houston) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Logged in as: tam2003


In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
os.chdir("/content")
!pip install -q transformers datasets

import json, random
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer

In [5]:
!pip install -q transformers datasets peft bitsandbytes flash-attn

MODEL = "Qwen/Qwen3-Coder-30B-A3B-Instruct"
DATASET = "/content/Finetuning-Qwen3/data/all_hybrid.jsonl"
DATA_COLUMN = "content"

SEQ_LENGTH = 4096

# Training arguments
MAX_STEPS = 200
NUM_EPOCHS = 5
BATCH_SIZE = 12
GR_ACC_STEPS = 2
LR = 2e-4                     # learning_rate
WARMUP_RATIO = 0.03
WEIGHT_DECAY = 0.05
LR_SCHEDULER_TYPE = "cosine"  # lr_scheduler_type
WEIGHT_DECAY = 0.05  # weight_decay
NUM_WARMUP_STEPS = 100  # num_warmup_steps
EVAL_FREQ = 25
SAVE_FREQ = 50
LOG_FREQ = 10
OUTPUT_DIR = "peft-FT-3-Coder-30b-v3"  # output_dir
BF16 = True  # bf16
FP16 = False  # no_fp16

# FIM trasformations arguments
FIM_RATE = 0.25  # fim_rate
FIM_SPM_RATE = 0.5  # fim_spm_rate


# LORA
LORA_R = 16  # lora_r
LORA_ALPHA = 32  # lora_alpha
LORA_DROPOUT = 0.05  # lora_dropout
LORA_TARGET_MODULES = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

# bitsandbytes config
USE_NESTED_QUANT = True  # use_nested_quant
BNB_4BIT_COMPUTE_DTYPE = "bfloat16"  # bnb_4bit_compute_dtype

SEED = 42

random.seed(SEED)

wandb.init(
    project="qwen3coder-finetune-fp16",
    name=f"run-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
)
wandb.define_metric("train/global_step")
wandb.define_metric("train/*", step_metric="train/global_step")
wandb.define_metric("eval/*",  step_metric="train/global_step")


In [6]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    logging,
    set_seed,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    TrainerCallback
)

In [7]:
os.chdir("/content/Finetuning-Qwen3")
from model.util import chars_token_ratio, split_by_filetype
os.chdir("/content")

In [8]:
import torch
from tqdm import tqdm

torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

tokenizer = AutoTokenizer.from_pretrained(
    MODEL,
    padding_side="right",
    add_eos_token=True,
    add_bos_token=True,
    trust_remote_code=True,
    use_fast=False,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset("json", data_files={"data": DATASET})["data"]

unsupervised = dataset.filter(lambda x: x["set"] == "unsupervised")

cryptol = unsupervised.filter(lambda x: x["filetype"] == "cry")
saw = unsupervised.filter(lambda x: x["filetype"] == "saw")
text = unsupervised.filter(lambda x: x["filetype"] == "txt")

cryptol_split = cryptol.train_test_split(test_size=0.1, seed=SEED, shuffle=True)
saw_split = saw.train_test_split(test_size=0.1, seed=SEED, shuffle=True)
text_split = text.train_test_split(test_size=0.1, seed=SEED, shuffle=True)

train_ds = concatenate_datasets([cryptol_split["train"], saw_split["train"], text_split["train"]])
eval_ds = concatenate_datasets([cryptol_split["test"], saw_split["test"], text_split["test"]])

print(train_ds[0].keys())
assert DATA_COLUMN in train_ds.column_names, f"Missing '{DATA_COLUMN}' in JSONL!"

print(eval_ds[0].keys())
assert DATA_COLUMN in eval_ds.column_names, f"Missing '{DATA_COLUMN}' in JSONL!"

print(len(train_ds), len(eval_ds))

chars_per_token = chars_token_ratio(train_ds, tokenizer, DATA_COLUMN)
print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

...

Filter:   0%|          | 0/998 [00:00<?, ? examples/s]

Filter:   0%|          | 0/747 [00:00<?, ? examples/s]

Filter:   0%|          | 0/747 [00:00<?, ? examples/s]

Filter:   0%|          | 0/747 [00:00<?, ? examples/s]

dict_keys(['filename', 'filetype', 'content', 'variant', 'set'])
dict_keys(['filename', 'filetype', 'content', 'variant', 'set'])
671 76
The character to token ratio of the dataset is: 2.26


In [9]:
from model.util import *

In [10]:
from model.constantLengthDataset import ConstantLengthDataset
import torch


train_dataset = ConstantLengthDataset(
        tokenizer=tokenizer,
        dataset=train_ds,
        infinite=False,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        overlap_ratio=0.125,
        seed=SEED,
        already_tokenized=False,
)
eval_dataset = ConstantLengthDataset(
        tokenizer=tokenizer,
        dataset=eval_ds,
        infinite=False,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        overlap_ratio=0.125,
        seed=SEED,
        already_tokenized=False,
)

[INFO] Empirical dataset length = 260 sequences (~929585 tokens)
[INFO] Empirical dataset length = 28 sequences (~100140 tokens)


In [11]:
import os, gc, torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
gc.collect(); torch.cuda.empty_cache()

In [12]:
! pip install -U bitsandbytes

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.tuners.lora import LoraLayer

names = ["<|fim_prefix|>", "<|fim_middle|>", "<|fim_suffix|>", "<|fim_pad|>"]
ids = [tokenizer.convert_tokens_to_ids(t) for t in names]
print(dict(zip(names, ids)))
print("additional_special_tokens:", tokenizer.special_tokens_map.get("additional_special_tokens"))

# 4-bit quantization
compute_dtype = getattr(torch, BNB_4BIT_COMPUTE_DTYPE)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=USE_NESTED_QUANT,
)

base = AutoModelForCausalLM.from_pretrained(
        MODEL,
        load_in_8bit=False,
        quantization_config=bnb_config,
        dtype=torch.bfloat16,
        device_map="auto",
        use_cache=True,
        trust_remote_code=True,
        attn_implementation="flash_attention_2",
)

base = prepare_model_for_kbit_training(base)

{'<|fim_prefix|>': 151659, '<|fim_middle|>': 151660, '<|fim_suffix|>': 151661, '<|fim_pad|>': 151662}
additional_special_tokens: ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']


config.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model-00002-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

model-00014-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00015-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00016-of-00016.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [13]:
blk = base.model.layers[0]           # Llama/Qwen-style
print("ATTN:", blk.self_attn)         # has q_proj, k_proj, v_proj, o_proj
print("MLP:", blk.mlp)
target_modules = LORA_TARGET_MODULES

ATTN: Qwen3MoeAttention(
  (q_proj): Linear4bit(in_features=2048, out_features=4096, bias=False)
  (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
  (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
  (o_proj): Linear4bit(in_features=4096, out_features=2048, bias=False)
  (q_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
  (k_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
)
MLP: Qwen3MoeSparseMoeBlock(
  (gate): Linear4bit(in_features=2048, out_features=128, bias=False)
  (experts): ModuleList(
    (0-127): 128 x Qwen3MoeMLP(
      (gate_proj): Linear4bit(in_features=2048, out_features=768, bias=False)
      (up_proj): Linear4bit(in_features=2048, out_features=768, bias=False)
      (down_proj): Linear4bit(in_features=768, out_features=2048, bias=False)
      (act_fn): SiLUActivation()
    )
  )
)


In [14]:
# Set up lora
from peft import PeftModel

peft_config = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

model = PeftModel.from_pretrained(base, "tam2003/peft-FT-3-Coder-30b-v3", is_trainable=True) # If your just trying to test/train the model.

# model = get_peft_model(base, peft_config) # If you want to continue training.

model.print_trainable_parameters()

adapter_config.json:   0%|          | 0.00/945 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/3.38G [00:00<?, ?B/s]

trainable params: 843,841,536 || all params: 31,375,964,160 || trainable%: 2.6895


In [15]:
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [16]:
train_ds.start_iteration = 0

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    gradient_accumulation_steps=GR_ACC_STEPS,
    gradient_checkpointing=False,
    learning_rate=LR,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    warmup_ratio=WARMUP_RATIO,
    weight_decay=WEIGHT_DECAY,
    fp16=True,
    bf16=False,
    optim="paged_adamw_8bit",
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to=["wandb"],
    push_to_hub=True,
    dataloader_drop_last=False,
    dataloader_num_workers=2,
    include_tokens_per_second=True,
    max_grad_norm=0.3,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    dataloader_pin_memory=True,
    save_safetensors=True,
    seed=SEED,
)

In [17]:
import math
from model.util import PerplexityCallback, EpochToDatasetCallback
from time import time
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,         # required for early stopping
    tokenizer=tokenizer,
    data_collator=collator,
    callbacks=[
        PerplexityCallback(),
        EpochToDatasetCallback(train_dataset),
        EarlyStoppingCallback(
            early_stopping_patience=2,
            early_stopping_threshold=0.0
        ),
    ],
)

train_start_time = time()
print("Training...")
trainer.train(resume_from_checkpoint=False)
train_end_time = time()
print(f"Training completed in {train_end_time - train_start_time:.2f} seconds.")
eval_results = trainer.evaluate()
wandb.finish()
eval_loss = eval_results["eval_loss"]
perplexity = math.exp(eval_loss)
print(f"Eval loss = {eval_loss:.2f}, Perplexity = {perplexity:.2f}")

/tmp/ipython-input-1719396369.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 151643}.


Training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Casting fp32 inputs back to torch.float16 for flash-attn compatibility.


Epoch,Training Loss,Validation Loss
1,1.105600,0.902883
2,0.952600,0.910360
3,0.828900,0.937576


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Training completed in 10074.94 seconds.


eval/loss,▁▁▃▃██▁▁
eval/perplexity,▁▂█▁
eval/runtime,▁▂█▅
eval/samples_per_second,██▁█
eval/steps_per_second,██▁█
train/epoch,▁▁▁▅▅▅██████
train/global_step,▁▁▁▅▅▅██████
train/grad_norm,▁▃█
train/learning_rate,█▅▁
train/loss,█▄▁
+1,...


Eval loss = 0.90, Perplexity = 2.47


In [18]:
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...-30b-v3/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

  ...adapter_model.safetensors:   1%|1         | 41.9MB / 3.38GB            

CommitInfo(commit_url='https://huggingface.co/tam2003/peft-FT-3-Coder-30b-v3/commit/ef6efa5d2a89aff28efccb9cdc5b7bc10d7a8a2c', commit_message='End of training', commit_description='', oid='ef6efa5d2a89aff28efccb9cdc5b7bc10d7a8a2c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tam2003/peft-FT-3-Coder-30b-v3', endpoint='https://huggingface.co', repo_type='model', repo_id='tam2003/peft-FT-3-Coder-30b-v3'), pr_revision=None, pr_num=None)

In [19]:
from google.colab import runtime

runtime.unassign()